<a href="https://colab.research.google.com/github/bxcharlie/study/blob/main/12_%E5%BA%94%E7%94%A8_LoRA_%E5%88%B0%E5%A4%A7%E6%A8%A1%E5%9E%8B%E7%9A%84%E7%AE%80%E5%8D%95%E7%A4%BA%E4%BE%8B%EF%BC%88PEFT%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 简介

> 指导文章：[14. PEFT：在大模型中快速应用 LoRA](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/14.%20PEFT：在大模型中快速应用%20LoRA.md)

## 安装必要的库

In [ ]:
!pip install transformers peft

## 加载预训练模型

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 加载预训练的 GPT-2 模型和分词器
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

## 使用 PEFT 应用 LoRA

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

# 配置 LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # 任务类型：因果语言模型
    inference_mode=False,          # 推理模式关闭，以进行训练
    r=8,                           # 低秩值 r
    lora_alpha=32,                 # LoRA 的缩放因子
    lora_dropout=0.1,              # Dropout 概率
)

# 将 LoRA 应用到模型中
model = get_peft_model(model, lora_config)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


## 查看当前模型架构

In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2SdpaAttention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_

## 查看增加的参数量

应用 LoRA 后，我们一般都希望了解模型参数量的变化，它的计算其实很简单，

### 理论计算

对于每个应用了 LoRA 的层，增加的参数量为：

$$
\text{增加的参数量} = r \times (\text{输入维度} + \text{输出维度})
$$

- **`r`**：LoRA 的低秩值。
- **输入维度**：层的输入特征数。
- **输出维度**：层的输出特征数。

### 使用 PEFT 查看参数

`peft` 提供了查看模型参数的便捷方法：

In [ ]:
# 查看 LoRA 模块
model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.2364


In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        all_params += num_params
        if param.requires_grad:
            trainable_params += num_params
    print(f"可训练参数量: {trainable_params}")
    print(f"总参数量: {all_params}")
    print(f"可训练参数占比: {100 * trainable_params / all_params:.2f}%")

print_trainable_parameters(model)

可训练参数量: 294912
总参数量: 124734720
可训练参数占比: 0.24%


## 准备数据并进行微调

假设你已经有了训练数据集 `train_dataset`，下面是一个简单的样例代码。

In [ ]:
from transformers import Trainer, TrainingArguments

# 定义训练参数
training_args = TrainingArguments(
    output_dir='./results',         # 模型保存和日志输出的目录路径
    num_train_epochs=3,             # 训练的总轮数（epochs）
    per_device_train_batch_size=16, # 每个设备（如GPU或CPU）上的训练批次大小，16表示每次输入模型的数据数量
    learning_rate=5e-5,             # 学习率
    logging_steps=10,               # 每隔多少步（steps）进行一次日志记录
    save_steps=100,                 # 每隔多少步保存模型
)

# 创建 Trainer
trainer = Trainer(
    model=model,                    # 训练的模型对象，需要事先加载好
    args=training_args,             # 上面定义的训练参数配置
    train_dataset=train_dataset,    # 需要对应替换成已经处理过的dataset
)

# 开始训练
trainer.train()

NameError: name 'train_dataset' is not defined

### 保存和加载 LoRA 微调的模型

训练完成后，你可以保存或者加载 LoRA 微调的参数，下面是个简单的示例。

In [ ]:
# 保存 LoRA 参数
model.save_pretrained('./lora_model')

In [ ]:
# 加载原始模型
base_model = AutoModelForCausalLM.from_pretrained("gpt2")

# 加载 LoRA 参数
from peft import PeftModel

model = PeftModel.from_pretrained(base_model, './lora_model')

### 合并 LoRA 权重并卸载 PEFT 包装

在完成微调后，可以使用 `merge_and_unload()` 将 LoRA 的权重合并回原始模型。这在部署和推理阶段非常有用，因为这样可以：

- **减少依赖**：合并后，模型成为标准的 `transformers` 模型，不再需要 `peft` 库。
- **提高推理效率**：减少了额外的计算开销，推理速度可能会有所提升。
- **简化模型保存和加载**：不需要分别保存基础模型和 LoRA 参数。

运行下面的代码：

In [ ]:
# 对比合并前后的模型
print("合并前的模型结构：")
print(model)

# 合并并卸载 LoRA 权重
model = model.merge_and_unload()

print("合并后的模型结构：")
print(model)

合并前的模型结构：
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2SdpaAttention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
            

你应该注意到，合并后模型的 LoRA 层将被去除。

现在，你可以像保存普通模型一样保存：

In [ ]:
# 保存合并后的模型
model.save_pretrained('./merged_model')
tokenizer.save_pretrained('./merged_model')

('./merged_model/tokenizer_config.json',
 './merged_model/special_tokens_map.json',
 './merged_model/vocab.json',
 './merged_model/merges.txt',
 './merged_model/added_tokens.json',
 './merged_model/tokenizer.json')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 加载合并后的模型
tokenizer = AutoTokenizer.from_pretrained('./merged_model')
model = AutoModelForCausalLM.from_pretrained('./merged_model')

# 进行推理
inputs = tokenizer("Hello, World！", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Hello, World！

I'm sorry, World！

I'm


**注意**：

- **不可逆操作**：合并操作是不可逆的。如果你之后还需要进一步微调 LoRA 参数，需确保在合并前备份模型。
- **无需 PEFT 库**：合并后的模型不再包含 LoRA 适配器（Adapter）的信息，因此在加载时无需使用 `PeftModel`。

## 可能的错误及解决方案（TypeError: Expected state_dict to be dict-like...）

在使用 `PEFT` 和 `LoRA` 进行模型微调和保存加载时，可能会遇到如下错误：

```
TypeError: Expected state_dict to be dict-like, got <class 'peft.peft_model.PeftModel'>.
```

### 错误原因

一般是因为混合使用不同的保存和加载方式，这个错误不局限于 `PeftModel`，问题出在你用 `torch.save(model)` 保存整个模型却用 `load_state_dict()` 去加载，注意模型加载和保存的一致性。

### 错误重现

下面我们来复现它，看是不是和你的操作一致（这里以 `PeftModel` 举例）：

#### 1. 错误地保存整个 `PeftModel` 对象而不是其 `state_dict`：

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

# 加载预训练模型和分词器
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

# 配置 LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

# 应用 LoRA
model = get_peft_model(model, lora_config)

# 错误地保存整个 PeftModel 对象
torch.save(model, './model')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


#### 2. 加载时传入 `PeftModel` 对象

In [ ]:
# 初始化模型
model = AutoModelForCausalLM.from_pretrained("gpt2")

# 错误地加载模型，期望接收 state_dict 但实际加载了整个模型对象
model.load_state_dict(torch.load('./model'))  # 这里会报错

<ipython-input-14-eee0690e3e0a>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./model'))  # 这里会报错


TypeError: Expected state_dict to be dict-like, got <class 'peft.peft_model.PeftModelForCausalLM'>.

### 解决方法

确保你保存和加载的对象是一致的：

- `torch.save(model, '...')` 对应于 `torch.load(model, '...')`。
- `torch.save(model.state_dict(), '...')` 对应于 `model.load_state_dict(torch.load('...'))`

## 一个非常刁钻的错误：使用 get_peft_model() 后再导入 LoRA

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from copy import deepcopy
from peft import get_peft_model, LoraConfig, PeftModel

# 固定随机数种子，确保结果可复现
torch.manual_seed(42)

# 定义一个简单的线性模型
class LinearModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

# 实例化线性模型
model = LinearModel(input_size=10, output_size=1)

# 在应用 LoRA 之前深拷贝原始模型，确保后续公平比较
original_model = deepcopy(model)

# 配置 LoRA 参数
config = LoraConfig(
    inference_mode=False,
    r=4,
    lora_alpha=16,
    target_modules=['linear'],
)

# 将 LoRA 应用到模型中
lora_model = get_peft_model(model, config)

# 定义一个简单的损失函数和优化器
criterion = nn.MSELoss()
optimizer = Adam(lora_model.parameters(), lr=1e-3)

# 生成一些模拟的训练数据
input_data = torch.randn(100, 10)  # 100 个样本，每个样本有 10 个特征
target_data = torch.randn(100, 1)  # 对应的目标值

# 训练一个回合
lora_model.train()
for epoch in range(1):  # 训练 1 个回合
    optimizer.zero_grad()
    outputs = lora_model(input_data)
    loss = criterion(outputs, target_data)
    loss.backward()
    optimizer.step()

# 训练后保存 LoRA 权重
lora_model.save_pretrained('linear_lora_model')

# 方法 1：先使用 get_peft_model，再加载 LoRA 权重
model1 = PeftModel.from_pretrained(get_peft_model(deepcopy(original_model), config), 'linear_lora_model')

# 方法 2：直接加载 LoRA 权重
model2 = PeftModel.from_pretrained(deepcopy(original_model), 'linear_lora_model')

# 生成相同的输入数据以进行输出比较
test_input = torch.randn(1, 10)

# 比较四个模型的输出（原始模型，LoRA，方法1，方法2）
def compare_model_outputs(input_data):
    # 原始模型
    original_output = original_model(input_data)
    print("原始模型输出:", original_output.detach().numpy())

    # 训练后的 LoRA 模型
    lora_output = lora_model(input_data)
    print("训练后的 LoRA 模型输出:", lora_output.detach().numpy())

    # 方法 1：先使用 get_peft_model，再加载 LoRA
    output1 = model1(input_data)
    print("方法 1（先使用 get_peft_model，再加载 LoRA）输出:", output1.detach().numpy())

    # 方法 2：直接加载 LoRA
    output2 = model2(input_data)
    print("方法 2（直接加载 LoRA）输出:", output2.detach().numpy())

    if torch.allclose(original_output, output1):
        print("\n原始模型和方法 1 输出相同。")
    if torch.allclose(lora_output, output2):
        print("训练后的 LoRA 模型和方法 2 输出相同。\n")

# 比较两个模型的参数
def compare_params(m1, m2):
    for (n1, p1), (n2, p2) in zip(m1.named_parameters(), m2.named_parameters()):
        if n1 != n2 or not torch.allclose(p1, p2):
            print(f"参数不匹配: \n{n1}\n{n2}")
            return False
    return True

# 比较四个模型的输出
compare_model_outputs(test_input)

# 检查方法 1 和方法 2 的参数是否一致
if compare_params(model1, model2):
    print("方法 1 和方法 2 的 LoRA 模型参数一致！")
else:
    print("方法 1 和方法 2 的 LoRA 模型参数不一致！")

原始模型输出: [[-0.03600371]]
训练后的 LoRA 模型输出: [[-0.03428639]]
方法 1（先使用 get_peft_model，再加载 LoRA）输出: [[-0.03600371]]
方法 2（直接加载 LoRA）输出: [[-0.03428639]]

原始模型和方法 1 输出相同。
训练后的 LoRA 模型和方法 2 输出相同。

参数不匹配: 
base_model.model.base_model.model.linear.base_layer.weight
base_model.model.linear.base_layer.weight
方法 1 和方法 2 的 LoRA 模型参数不一致！
